# Essential TitaniumCloud file analysis and reputation API-s
This notebook contains specific examples for the essential TitaniumCloud file reputation and analysis-related API-s.
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Covered API classes
This notebook covers examples for the following API classes:
- **FileReputation** (*TCA-0101 - File Reputation (Malware Presence)*)
- **AVScanners** (*TCA-0103 - Historic Multi-AV Scan Records*)
- **FileAnalysis** (*TCA-0104 - File Analysis - Hash*)
- **RHA1FunctionalSimilarity** (*TCA-0301 - RHA Functional Similarity (Group by RHA1)*)

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

### 1. Importing the required classes
First, we will import the required API classes from the ticloud module.

In [13]:
from ReversingLabs.SDK.ticloud import FileReputation, AVScanners, FileAnalysis, RHA1FunctionalSimilarity

### 2. Loading the credentials
Next, we will load our TitaniumCloud credentials from the local `ticloud_credentials.json` file.
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**

Also, we will store the SHA-1 hash of a file we want to investigate into a variable called `FILE_HASH`.

In [14]:
import json


CREDENTIALS = json.load(open("ticloud_credentials.json"))
USERNAME = CREDENTIALS.get("username")
PASSWORD = CREDENTIALS.get("password")

FILE_HASH = "21841b32c6165b27dddbd4d6eb3a672defe54271"

### 3. Fetching the file reputation
Next, we will create our first object out of the FileReputation class: file_reputation

In [15]:
file_reputation = FileReputation(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

We will now use our `file_reputation` object to do a reputation lookup for the hash of our potentially malicious file.

In [16]:
reputation = file_reputation.get_file_reputation(
    hash_input=FILE_HASH
)

reputation_text = reputation.text
print(reputation_text)

{"rl": {"malware_presence": {"status": "MALICIOUS", "sha1": "21841b32c6165b27dddbd4d6eb3a672defe54271", "threat_level": 5, "classification": {"platform": "Win32", "type": "Ransomware", "is_generic": false, "family_name": "Tox"}, "scanner_percent": 94.59459686279297, "threat_name": "Win32.Ransomware.Tox", "scanner_match": 35, "last_seen": "2023-07-12T12:50:36", "reason": "antivirus", "scanner_count": 37, "query_hash": {"sha1": "21841b32c6165b27dddbd4d6eb3a672defe54271"}, "first_seen": "2015-05-30T22:04:00", "sha256": "2f6edf41016e97c58f9de01aa4cc66c9c7fe7dae23fe72e50a69cbd221f55346", "trust_factor": 5, "md5": "3133c2231fcee5d6b0b4c988a5201da1"}}}


Now we have the file reputation response stored in the `reputation` variable and also printed out in the console in text form.
Among other information, from the file reputation response we can see the following:
- This file is malicious - `"status": "MALICIOUS"`
- Its threat name is Win32.Ransomware.Tox - `"threat_name": "Win32.Ransomware.Tox"`
- There is a certain number of AV scanners, out of a total number of available AV scanners, that declared this file as harmful - `"scanner_match": 35`

### 4. Fetching the AV scanner results
These results provide malware detection verdicts of multiple antivirus scanners from the TitaniumCloud.
To see which AV scanners declared the file as harmful, we can use the AVScanners class.
To do so, we will again create an object:

In [17]:
av_scanners = AVScanners(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

Now that we have our `av_scanners` object, we can fetch the AV scanner results from the API.

In [18]:
scanner_results = av_scanners.get_scan_results(
    hash_input=FILE_HASH
)

results_text = scanner_results.text
print(results_text)

{"rl": {"sample": {"sha1": "21841b32c6165b27dddbd4d6eb3a672defe54271", "first_seen_on": "2015-05-30T22:04:00", "last_scanned_on": "2023-07-12T12:50:00", "last_seen_on": "2023-07-12T12:50:00", "single_scan": false, "first_scanned_on": "2015-05-30T22:04:00", "sample_type": "PE32 executable (GUI) Intel 80386 (stripped to external PDB), for MS Windows, UPX compressed", "sample_size": 636416, "xref": [{"scanner_match": 35, "scanner_count": 38, "scanners": [{"timestamp": "2023-07-12T09:33:00", "version": "ahnscan-console.zip", "name": "ahnlab"}, {"timestamp": "2023-07-12T09:33:00", "version": "ahnscan-console.zip", "name": "ahnlab_online"}, {"timestamp": "2023-07-12T11:30:00", "version": "vdf_fusebundle.zip", "name": "antivir"}, {"timestamp": "2023-07-12T11:35:00", "version": "avast_stream.zip", "name": "avast"}, {"timestamp": "2023-07-12T11:31:00", "version": "bitdefender_pack.rar", "name": "bitdefender"}, {"timestamp": "2023-07-12T11:30:00", "version": "carbonblack_pack.rar", "name": "carb

To see which scanners were available, navigate to `"rl" -> "sample -> "xref" -> "scanners"`  
To see the verdict returned by each scanner, navigate to `"rl" -> "sample -> "xref" -> "results"`  

Other useful information included in the AV scanners report:
- `"sample_type": "PE32 executable (GUI) Intel 80386 (stripped to external PDB), for MS Windows, UPX compressed"`
- `"sample_size": 636416`

### 5. Fetching the file analysis results
To view the static analysis details of our malicious file, we can use the FileAnalysis class:

In [19]:
analysis = FileAnalysis(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

analysis_results = analysis.get_analysis_results(
    hash_input=FILE_HASH
)

analysis_text = analysis_results.text
print(analysis_text)

{"rl":{"sample":{"sha1":"21841b32c6165b27dddbd4d6eb3a672defe54271","crc32":"8704451d","md5":"3133c2231fcee5d6b0b4c988a5201da1","sha256":"2f6edf41016e97c58f9de01aa4cc66c9c7fe7dae23fe72e50a69cbd221f55346","sha384":"e0b7bf0ad928500ee1dc06f8cbe035e663eaf546bb4b5217706706ba12c50ab6a24e1e858dae9a5ce0f7673bdb5621be","sha512":"205ece960784bff6fdbd0d5a1ebad4fddeab6751728d5be2e0b5d91742d520df0c5d04fd3b9e67372c35cb0859d794b7d22ea78786669a4bd5725e814548143f","ripemd160":"d26f686b6af13b9073f77a1ba5a7b610934dc625","ssdeep":"12288:UxvYm8UX7FkiYiHSbhy783clwXqaAQWzRTChYl:+vY0LFrYi0s7w6a/Wzl","sample_size":636416,"relationships":{"container_sample_sha1":["50267628309d0e320d6ed25b198bb9a9a6181535","0656564814da810938c100e7fef5bf14cc8fa691","21841b32c6165b27dddbd4d6eb3a672defe54271","f0d94e01b7c39bcd7fbf901811bfc7d8ea49bc11"],"parent_sample_sha1":["0656564814da810938c100e7fef5bf14cc8fa691","f0d94e01b7c39bcd7fbf901811bfc7d8ea49bc11","50267628309d0e320d6ed25b198bb9a9a6181535"]},"analysis":{"entries":[{"reco

By reading the analysis results, we can get the following useful information:
- `"sample_size": 636416`
- `"tc_report"` - a static analysis report created by TitaniumCore
    - `"file_type": "PE"`
    - `"file_subtype": "Exe"`
    - `"story"` - a verbose and human-readable description of this file
    - `"dynamic_analysis"` - dynamic analysis results from various cloud sandboxes


### 6. Getting the hashes of functionally similar files
Additionally, we can send use the RHA1FunctionalSimilarity class to get a list of files that are functionally similar to the malicious file that we are analyzing.  
Each file is represented by its SHA-1 hash.

In [ ]:
similarity = RHA1FunctionalSimilarity(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

similarity_results = similarity.get_similar_hashes(
    hash_input=FILE_HASH
)

similarity_text = similarity_results.text
print(similarity_text)

This gives us a report with only the first page of results. If we want a **complete** list of similar files, we need to use the `get_similar_hashes_aggregated` method.  
The full list is returned as a Python `list` type, rather than a response object.
Example:

In [ ]:
all_pages = similarity.get_similar_hashes_aggregated(
    hash_input=FILE_HASH
)

print(all_pages)


### 7. Advanced
**Additional parameters** in that method give us the following options:
- `extended_results` - show the extended results
- `classification` - return only files of this classification
- `max_results` - the maximum number of files we want
- `results_per_page` - the number of files per each page (request) we want, until we reach the `max_results`

Example:

In [ ]:
all_pages = similarity.get_similar_hashes_aggregated(
    hash_input=FILE_HASH,
    extended_results=True,
    classification="MALICIOUS",
    max_results=300,
    results_per_page=50
)

print(all_pages)

This example will return a maximum of 300 files in a list, with 50 files per page (request). Also, it will return only malicious files with extended information.